# Projekt1 - Beizen der Schweiz

## APIs, Requests, Visualisierung

#### Nenn mir deine Stammbeiz und ich sage dir, wo du wohnst

Welches sind die vorherrschenden Beizennamen pro Kanton? Recherche mit der Swisscom Directories AG API.
Mein persönlicher API Schlüssel: 590ea8961488f9a63dbf95a7b7883805

https://tel.search.ch/api/help


BEISPIEL: https://tel.search.ch/api/?was=john+meier&key=Ihr Schlüssel

# NACHTRAG: DIE API DES GRAUENS???

## Lieber Barnaby,

gestern habe ich das Abfrage-Limit der API erreicht und via Upload der ersten 200 Adressen auf unseren Storytelling-Server meinen Code und Schlaufen kontrolliert. Heute ist die API wieder offen, ich komme jedoch nicht über eine Abfrage von maximal 400 Adressen (der Kanton Zürich hat jedoch 4198 Beizen) hinaus. Mein Kollege Simon Huwyler sieht auch keine Lösung und ich erst recht nicht... ; ))) 
Ausser natürlich, ich scrape die Website und vergesse die API. Zuvor möchte ich dir jedoch gerne mein Notebook und Code zeigen. Hast du eine clevere Idee, wie ich mein Problem lösen könnte? 

Ich habe auch meine ganzen Arbeitsschritte ausdokumentiert...

Herzlichen Dank und liebe Grüsse

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

%matplotlib inline

import json

# Als Erstes alles importieren, was ich glaube zu brauchen...

### Dann hole ich mir die Website, mit deren API ich arbeite

Anhand der API Spezifikationen kann ich mir die genauen Parameter raussuchen, nach denen ich suchen will.
Hier **was** (für Restaurant) , **wo** (für Kanton), und da TelSearch die requests auf 200 beschränkt hat die **maxnum** und **pos** (um eine Schlaufe bilden zu können für alle Einträge die mehr als 200 (maxnum) sind.

In [4]:
"https://tel.search.ch/api/?was=Restaurant&key=590ea8961488f9a63dbf95a7b7883805"

'https://tel.search.ch/api/?was=Restaurant&key=590ea8961488f9a63dbf95a7b7883805'

Durch die Spezifikationen passe ich meine URL an und speichere sie gleich in eine Variable. Das unten heisst so viel wie alle Einträge in Restaurants des Kantons Schaffhausen, die ersten 200 (maxnum!!! GRRRR), bitte.

In [3]:
url = "https://tel.search.ch/api/?was=Restaurant&wo=SH&key=590ea8961488f9a63dbf95a7b7883805&maxnum=200"

Mit **requests** rufe ich die Website auf, auch dies wird gleich in eine Variable **response** gepackt.

In [5]:
response = requests.get(url)

Was spuckt er aus? Erste Kontrolle, ob mein request richtig verpackt war. Und das dann gleich noch als Textausgabe.

In [9]:
response

<Response [429]>

In [11]:
response.text

'Too many requests, try again tomorrow.'

Nun verpacke ich die URL in verschiedene Variablen um damit übersichtlicher arbeiten zu können. Dies gibt dann die nächste Variable **url_sh** und meine nächste Kontrolle, ob die Anfrage richtig gestellt war.

In [12]:
basis = "https://tel.search.ch/api/?"
was = "was=Restaurant&"
wo = "wo=SH&"
maxnum = "maxnum=200"
key = "&key=590ea8961488f9a63dbf95a7b7883805"

In [13]:
url_sh = basis + was + wo + maxnum + key

In [14]:
requests.get(url_sh)

<Response [429]>

In [15]:
requests.get(url_sh).text

'Too many requests, try again tomorrow.'

Ich will mir die zusammengesetzte Website-URL nochmals anschauen...

In [17]:
print(url_sh)

https://tel.search.ch/api/?was=Restaurant&wo=SH&maxnum=200&key=590ea8961488f9a63dbf95a7b7883805


Was ist da drin? Nach was kann ich das dann übersichtlicher darstellen? Bevor ich in **response.json()** gehe, schaue ich mir den Header an.

In [18]:
requests.get(url_sh).headers

{'Server': 'Apache', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 10 Oct 2018 15:29:47 GMT', 'Access-Control-Allow-Origin': '*', 'Connection': 'Keep-Alive', 'Set-Cookie': 'myosotis=163b7d4483f3652bfd7d81489be2bff6; expires=Tue, 19-Jan-2038 03:14:07 GMT; Max-Age=608298260; path=/; domain=.search.ch; HttpOnly', 'Content-Length': '38'}

Nun wurde es richtig übel: Das json-Ding, was wir bis anhin immer verwendet haben, funktioniert nicht. Es gibt eine Fehlermeldung aus (siehe unten).
Dann geht das grosse Troubleshooten los: wieso? alles ist wie im Kurs gemacht! Was heisst RESTful API? Was steht in der request-documentary? Eine schlaflose Nacht und ich weiss, ich brauche Hilfe...

In [ ]:
response.json()

Aber erst versuche ich, einfach weiter zu machen. Was passiert, wenn ich die ganze Sache einfach mal in BeautifulSoup packe? Brauche ich die blöde json-Darstellung gar nicht? Einfach mal ausprobieren... Ich hole mir eine Lösung aus den Schulunterlagen, wo ich das Gefühl habe, das könnte eventuell (die Hoffnung stirbt zuletzt) funktionieren.
Und oben hab ich immerhin schon gesehen, dass ich für die Restaurantnamen nach "entry" suchen sollte. Also...

In [19]:
r = requests.get(url_sh)

In [20]:
contents = r.text #Wandeln wir den Text in ein Format um, mit dem BeautifulSoup umgehen kann.
soup = BeautifulSoup(contents,'html.parser') #Geben wir das an BeautifulSoup weiter
entries = soup.find_all('entry') #Nun lesen wir Titel aus.
len(entries) #Schauen wir, wie lange die Titel sind.

0

### Arrrghhh... 

TelSearch spuckt kein json, sondern nur XML aus!!!!
Hilfe. Mit Simons Hilfe erarbeite ich einen Workaround und lerne einiges über xml. Eventuell brauche ich einen neuen Parser, der wird vorsorglich unten installiert.
Dazu weiss ich, dass ich alles in Pandas darstellen will. Also baue ich mir ein leeres Dataframe, wo ich dann wie in einen Container alles abfüllen kann. Ich brauche einen Boolean, verschiedene Variablen und vor allem durch die Beschränkung auf 200 Adressen pro Abfrage eine **Schleife!!!**, die mir die anderen Restaurants auch noch holt. Und weil ich das ganze ja für alle Restaurants der Schweiz haben will, nehme ich den Kanton (hier "SH") auch noch in eine Variable.
Wenn der Code unten funktioniert, brauche ich "nur" noch die Kantonsvariable zu ändern und kann das Ganze ziemlich speditiv abfragen und in ein CSV speichern.
Hat natürlich viel länger gedauert zu erarbeiten, als hier zu beschreiben ; )

Und ja, ich habe bereits die maximal erlaubte Anzahl Abfragen pro Tag hingekriegt und musste das XML-File der ersten 200 Abfragen auf unseren Server nehmen, um weiterarbeiten zu können...Doch damit konnte ich wenigstens die Funktionalität testen und kann nun Tag für Tag runterladen, so viel wie halt geht...

In [ ]:
!pip3 install --upgrade lxml
!pip3 install cssselect


In [23]:
kanton = "SH"

In [24]:
import lxml

In [31]:
df = pd.DataFrame({"Restaurantname":[]})
df.head()

,Restaurantname


In [33]:
hatrestaurant = True
counter = 1

ns_opensearch = {'openSearch': 'http://a9.com/-/spec/opensearchrss/1.0/'}

while hatrestaurant == True:
    
    print (str(counter))
    
    basis = "https://tel.search.ch/api/?"
    #basis = "https://devst.blick.ch/intern/priska/api.xml?"
    was = "was=Restaurant&"
    wo = "wo="+ kanton + "&"
    maxnum = "maxnum=200&"
    pos = "pos=" + str(counter * 200 - 200 + 1)
    key = "&key=590ea8961488f9a63dbf95a7b7883805"
    
    url_sh = basis + was + wo + maxnum + pos + key
    print(url_sh)
    
    r = requests.get(url_sh)
    soup = BeautifulSoup(r.text, 'html.parser')
    #print(soup.find("opensearch:totalresults").text)
    
    #Alle Entrys vom aktuellen file durchsuchen
    entries = soup.find_all("entry")
    for entry in entries:
        #print (entry.find("title").text)
        restaurantname = entry.find("title").text
        df = df.append({"Restaurantname": restaurantname},ignore_index=True)
    
    
    
    #Kontrolliert schleife
    if counter * 200 < int(soup.find("opensearch:totalresults").text):
        hatrestaurant = True
    else:
        hatrestaurant = False
    
    counter = counter + 1
    
print("finito")

1
https://tel.search.ch/api/?was=Restaurant&wo=SH&maxnum=200&pos=1&key=590ea8961488f9a63dbf95a7b7883805
2
https://tel.search.ch/api/?was=Restaurant&wo=SH&maxnum=200&pos=201&key=590ea8961488f9a63dbf95a7b7883805


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
df.to_csv("export_csv/export_" + kanton + ".csv", sep = ";" , index = False)

In [30]:
#df = pd.DataFrame({"Restaurantname":[]})
#entries = soup.find_all("entry")
#for entry in entries:
 #   print (entry.find("title").text)
    
  #  restaurantname = entry.find("title").text
    
   #df = df.append({"Restaurantname": restaurantname},ignore_index=True)

#das hier ist Ersatzcode, rauslöschen, wenn alles läuft!!!

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 1 columns):
Restaurantname    600 non-null object
dtypes: object(1)
memory usage: 4.8+ KB


In [28]:
df.head(10)

,Restaurantname
0,"Güterhof Restaurant, Bar, Sushi, Dine"
1,Restaurant Burg Hohenklingen
2,Restaurant 1584
3,Restaurant Rhyhalde
4,Restaurant Trattoria la Calabrisella
5,Restaurant Park am Rheinfall
6,Beckenburg das Restaurant
7,Restaurant Mühle
8,Restaurant Rossberghof
9,Restaurant Sommerlust


### Meine nächsten Schritte

Alle Kantone durch anpassen der Kantonsvariablen rausziehen und **sofort** als CSV abspeichern. Somit hoffe ich, nicht nochmals das API Limit von TelSearch zu erreichen.

Die Kantons-CSV einlesen und mit Pandas nach Häufigkeit vom Beizennamen listen, die Top5 sollten reichen.

Die Kantons-CSV einlesen und mit Pandas zu einem grossen Schweiz-File zusammenfügen. Welches sind die Top5-Beizennamen der Schweiz?

Dann nach prozentualer Häufigkeit listen. Der beliebteste Beizenname pro Kanton wie häufig wird er nach Prozent vergeben?

**Eventuell** schaffe ich es mein Wohnbevölkerungs-Excel mit den Beizen pro Kanton zu erweitern... Ansonsten nehme ich das aus meiner Excel-Vorlage. Wieviele Beizen pro 1000 EW/Kanton gibt es? Wo ist die relative höchste Beizendichte?

Und damit kann ich dann anfangen zu zeichnen... : )))) **Hurra, zeichnen!!!**